In [3]:
# define logging and working directory
from ProjectRoot import change_wd_to_project_root 
change_wd_to_project_root()

import SimpleITK as sitk
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
import shutil
import numpy as np
from scipy.spatial.transform import Rotation as R
from src.data.Preprocess import *
from src.utils.Notebook_imports import *


search for root_dir and set working directory
Working directory set to: /mnt/ssd/git/3d-mri-domain-adaption


# Resample to isotrop

This method creates strange artefacts, so far I dont know how to solve

In [4]:
ax_src = '/mnt/ssd/data/gcn/ax_sax_from_flo/ax3d/'
sax_src = '/mnt/ssd/data/gcn/ax_sax_from_flo/sax3d/'
ax_iso_dst = '/mnt/ssd/data/gcn/ax_sax_from_flo/ax3d_iso_linear/'
sax_iso_dst = '/mnt/ssd/data/gcn/ax_sax_from_flo/sax3d_iso_linear/'

In [5]:
# resample 3D image / mask into a fake isotrop resolution, use multithreadding, combine iso and orientation resample, everything in one step
import concurrent.futures
from concurrent.futures import as_completed
slice_first = 2

workers = 16
futures_ax_iso = set()
futures_sax_iso = set()
futures_ax2sax_iso = set()
futures_sax2ax_iso = set()

ax_files = sorted(glob.glob(ax_src+'*.nrrd', recursive = True))
sax_files = sorted(glob.glob(sax_src+'*.nrrd', recursive = True))
ensure_dir(ax_iso_dst)
ensure_dir(sax_iso_dst)

target_spacing = (1.5,1.5,1.5)
# spawn multiple threads or processes
with concurrent.futures.ThreadPoolExecutor(max_workers=workers) as executor:# resample ax towards sax and vize versa
    
    for ax_file, sax_file in zip(ax_files, sax_files):
        print('processing file: {}'.format(ax_file))
        reader1 = sitk.ImageFileReader()
        reader1.SetFileName(ax_file)
        ax_img = reader1.Execute()
        
        reader2 = sitk.ImageFileReader()
        reader2.SetFileName(sax_file)
        sax_img = reader2.Execute()
        
        # make a pseudo isotrop volume from both input images
        if 'img' in os.path.basename(ax_file):
            futures_ax_iso.add(executor.submit(transform_to_isotrop_voxels, ax_img, sitk.sitkLinear,target_spacing, ax_file))
            futures_sax_iso.add(executor.submit(transform_to_isotrop_voxels, sax_img, sitk.sitkLinear,target_spacing, sax_file))
        else:
            futures_ax_iso.add(executor.submit(max_thres_resample2_iso_label_img, ax_img,50, target_spacing, ax_file, sitk.sitkLinear))
            futures_sax_iso.add(executor.submit(max_thres_resample2_iso_label_img, sax_img,50, target_spacing, sax_file, sitk.sitkLinear))

# this part is sequential, we need only one writer
writer = sitk.ImageFileWriter()
for future in as_completed(futures_ax_iso):
    try:
        res = future.result()
        iso_resampled, file_path = res
        writer.SetFileName(os.path.join(ax_iso_dst,os.path.basename(file_path)))
        writer.Execute(iso_resampled)

    except Exception as e:
        logging.error(
            'Exception {} in datagenerator with: image: {} or mask: {}'.format(str(e)))
        
for future in as_completed(futures_sax_iso):
    try:
        res = future.result()
        iso_resampled, file_path = res
        writer.SetFileName(os.path.join(sax_iso_dst,os.path.basename(file_path)))
        writer.Execute(iso_resampled)

    except Exception as e:
        logging.error(
            'Exception {} in datagenerator with: image: {} or mask: {}'.format(str(e)))



In [21]:
ax_src = '/mnt/ssd/data/gcn/ax_sax_from_flo/ax3d_iso_linear/'
sax_src = '/mnt/ssd/data/gcn/ax_sax_from_flo/sax3d_iso_linear/'
dst_ax2sax = '/mnt/ssd/data/gcn/ax_sax_from_flo/sax2ax3d_iso/'
ax2sax_iso_dst = '/mnt/ssd/data/gcn/ax_sax_from_flo/ax2sax3d_iso_linear/'
sax2ax_iso_dst = '/mnt/ssd/data/gcn/ax_sax_from_flo/sax2ax3d_iso_linear/'

# Resample to other orientation

In [23]:
# resample 3D image / mask from axial view into sax view or vize versa, extend the transformed vol by 10 voxel along the target z axis
import concurrent.futures
from concurrent.futures import as_completed

workers = 16
futures_ax2sax = set()
futures_sax2ax = set()
ensure_dir(ax2sax_iso_dst)
ensure_dir(sax2ax_iso_dst)
# spawn multiple threads or processes
# transform the image of file_path with according to the direction of 
ax_files = sorted(glob.glob(ax_src+'*.nrrd', recursive = True))
sax_files = sorted(glob.glob(sax_src+'*.nrrd', recursive = True))
print('axial files: {}'.format(len(ax_files)))
print('sax files: {}'.format(len(sax_files)))
with concurrent.futures.ThreadPoolExecutor(max_workers=workers) as executor:
    
    for ax_file_path, sax_file_path in zip(ax_files, sax_files):
        
        reader1 = sitk.ImageFileReader()
        reader1.SetFileName(ax_file_path)
        ax_img = reader1.Execute()
        
        reader2 = sitk.ImageFileReader()
        reader2.SetFileName(sax_file_path)
        sax_img = reader2.Execute()
    
        if 'img' in os.path.basename(ax_file_path):
            futures_ax2sax.add(executor.submit(resample_direcion_origin_spacing_shift, ax_img, sax_img, (0,0,-10), sitk.sitkLinear, ax_file_path))
            futures_sax2ax.add(executor.submit(resample_direcion_origin_spacing_shift, sax_img, ax_img, (0,0,-10), sitk.sitkLinear, sax_file_path))

        else:
            futures_ax2sax.add(executor.submit(max_thres_resample2_label_img_shift, ax_img, sax_img, 50, (0,0,-10), ax_file_path))
            futures_sax2ax.add(executor.submit(max_thres_resample2_label_img_shift, sax_img, ax_img, 50, (0,0,-10), sax_file_path))

writer = sitk.ImageFileWriter()
for i, future in enumerate(as_completed(futures_ax2sax)):
    try:
        res = future.result()
        resampled, file_path = res
        writer.SetFileName(os.path.join(ax2sax_iso_dst,os.path.basename(file_path)))
        writer.Execute(resampled) 

    except Exception as e:
        logging.error(
            'Exception {} in datagenerator with: image: {} or mask: {}'.format(str(e)))
        
for i, future in enumerate(as_completed(futures_sax2ax)):
    try:
        res = future.result()
        resampled, file_path = res
        writer.SetFileName(os.path.join(sax2ax_iso_dst,os.path.basename(file_path)))
        writer.Execute(resampled) 

    except Exception as e:
        logging.error(
            'Exception {} in datagenerator with: image: {} or mask: {}'.format(str(e)))

axial files: 324
sax files: 324


# Manual tests, not needed currently

In [20]:
ax_src = '/home/rflo/master/all_with_7_worst_regi/ax3d/'
sax_src = '/home/rflo/master/all_with_7_worst_regi/sax3d/'
dst = '/home/rflo/master/all_with_7_worst_regi/testground/'

In [21]:
ensure_dir(dst)

In [ ]:
ax_src = sorted(glob.glob('/home/rflo/master/all_with_7_worst_regi/ax3d/'+'*.nrrd', recursive = True))[0]
sax_src = sorted(glob.glob('/home/rflo/master/all_with_7_worst_regi/sax3d/'+'*.nrrd', recursive = True))[0]

In [22]:
ax_src = sorted(glob.glob('/home/rflo/master/all_with_7_worst_regi/ax3d_iso_linear/'+'*.nrrd', recursive = True))[0]
sax_src = sorted(glob.glob('/home/rflo/master/all_with_7_worst_regi/sax3d_iso_linear/'+'*.nrrd', recursive = True))[0]

In [ ]:
ax_src = sorted(glob.glob('/home/rflo/master/all_with_7_worst_regi/ax3d_iso_linear/'+'*.nrrd', recursive = True))[28]
sax_src = sorted(glob.glob('/home/rflo/master/all_with_7_worst_regi/sax3d_iso_linear/'+'*.nrrd', recursive = True))[28]

In [23]:
print(ax_src)
print(sax_src)

/home/rflo/master/all_with_7_worst_regi/ax3d_iso_linear/0000-0HQQW4ZN_2007-05-23_ED_img.nrrd
/home/rflo/master/all_with_7_worst_regi/sax3d_iso_linear/0000-0HQQW4ZN_2007-05-23_ED_img.nrrd


In [24]:
from scipy.ndimage import affine_transform
#testing single pictures
ax_file_path = ax_src
sax_file_path = sax_src

reader1 = sitk.ImageFileReader()
reader1.SetFileName(ax_file_path)
ax_img = reader1.Execute()

reader2 = sitk.ImageFileReader()
reader2.SetFileName(sax_file_path)
sax_img = reader2.Execute()

In [25]:
for shift in range(-20,1,2):    
    resampled = resample_direcion_origin_spacing_shift(ax_img, sax_img, shift = (0,0,shift), interpolate=sitk.sitkLinear)
    writer = sitk.ImageFileWriter()
    writer.SetFileName(os.path.join(dst,'shift'+str((0,0,shift))+'.nrrd'))
    writer.Execute(resampled) 

In [ ]:
for shift in range(-20,21,5):    
    resampled = resample_direcion_origin_spacing_shift(ax_img, sax_img, shift = (shift,0,0), interpolate=sitk.sitkLinear)
    writer = sitk.ImageFileWriter()
    writer.SetFileName(os.path.join(dst,'shift'+str((shift,0,0))+'.nrrd'))
    writer.Execute(resampled) 
    resampled = resample_direcion_origin_spacing_shift(ax_img, sax_img, shift = (0,shift,0), interpolate=sitk.sitkLinear)
    writer = sitk.ImageFileWriter()
    writer.SetFileName(os.path.join(dst,'shift'+str((0,shift,0))+'.nrrd'))
    writer.Execute(resampled) 
    resampled = resample_direcion_origin_spacing_shift(ax_img, sax_img, shift = (0,0,shift), interpolate=sitk.sitkLinear)
    writer = sitk.ImageFileWriter()
    writer.SetFileName(os.path.join(dst,'shift'+str((0,0,shift))+'.nrrd'))
    writer.Execute(resampled) 

In [ ]:
"""
sitk_img1 = ax_img
sitk_img2 = sax_ixmg
    
label1_img1 = get_single_label_img(sitk_img1,1)
label2_img1 = get_single_label_img(sitk_img1,2)
label3_img1 = get_single_label_img(sitk_img1,3)
resampled_label1 = resample_img(label1_img1, sitk_img2)
resampled_label2 = resample_img(label2_img1, sitk_img2)
resampled_label3 = resample_img(label3_img1, sitk_img2)
"""
size = ax_img.GetSize()
spacing = ax_img.GetSpacing()
size_new = [int(s*p) for s,p in zip(size,spacing)]
size_new = tuple(size_new)

In [ ]:
resampler = sitk.ResampleImageFilter()
resampler.SetSize(size_new)
resampler.SetOutputSpacing((1,1,1))
resampler.SetOutputOrigin(ax_img.GetOrigin())
#resampler.SetInterpolator(sitk.sitkNearestNeighbor)
new_img = resampler.Execute(ax_img)

#sitk_img, size=(12, 256, 256), spacing=(8, 1.25, 1.25), interpolate=sitk.sitkNearestNeighbor


ax_array = sitk.GetArrayFromImage(new_img)
#ax_img.SetDirection(sax_img.GetDirection())
direction = sax_img.GetDirection()
direction = np.reshape(direction,(3,3))


#r = R.from_matrix(direction)
new_array = affine_transform(ax_array,direction)
new_img = sitk.GetImageFromArray(new_array)
new_img.SetOrigin(ax_img.GetOrigin())
new_img.SetDirection(sax_img.GetDirection())   

#resampled = max_thres_resample_label_img(ax_img, sax_img,50)
#resampled = percentage_resample_label_img(ax_img, sax_img,80)

writer = sitk.ImageFileWriter()
writer.SetFileName(os.path.join(dst,os.path.basename('img_ax_new.nrrd')))
writer.Execute(new_img) 
#writer = sitk.ImageFileWriter()
#writer.SetFileName(os.path.join(dst,os.path.basename('sax_0.nrrd')))
#writer.Execute(sax_img) 

#writer = sitk.ImageFileWriter()
#writer.SetFileName(os.path.join(dst,os.path.basename(ax_file_path)))
#writer.Execute(resampled) 

In [ ]:
png_path = '/home/rflo/master/data_search/ax'

In [ ]:
png_path = '/home/rflo/master/data_search/ax_to_sax/'

In [ ]:
from IPython.display import Image
from ipywidgets import interact
path_to_png = png_path
@interact
def show_png(i = (0, len(sorted(glob.glob(os.path.join(path_to_png,'*.png'))))-1)):
    f_name = sorted(glob.glob(os.path.join(path_to_png,'*.png')))[i]
    print(f_name)
    return Image(filename=f_name)

In [ ]:
ax_src = '/home/rflo/master/all_with_7_worst_regi/ax3d/'
sax_src = '/home/rflo/master/all_with_7_worst_regi/sax3d/'
dst = '/home/rflo/master/all_with_7_worst_regi/testground/'

In [ ]:
ax_src = '/home/rflo/master/all_with_7_worst_regi/ax3d_iso_linear/'
sax_src = '/home/rflo/master/all_with_7_worst_regi/sax3d_iso_linear/'
dst = '/home/rflo/master/all_with_7_worst_regi/testground/'

In [ ]:
ensure_dir(dst)

In [ ]:
src = ax_src

In [ ]:
#copy all msk into dst

for file_path in sorted(glob.glob(src+'*.nrrd', recursive = True)):
    if 'msk' in os.path.basename(file_path):
        shutil.copyfile(file_path,os.path.join(dst,os.path.basename(file_path)))
        